# `Example`: Training Binary Bernoulli Bandit Ticker Picker Agents
In this example, we create a group of `N` agents to assist us in selecting stocks for our portfolio $\mathcal{P}$. We will use the [$\epsilon$-Greedy Thompson Sampling algorithm](https://arxiv.org/abs/1707.02038) to solve a Multi-arm Binary Bandit problem. The choices of stocks that we can select from will be the bandits, and there are `K` of them in our choice set $\mathcal{C}$. We will feed daily return data to the group of `N` agents, who will learn which tickers return beyond a specified cutoff, such as the risk-free rate, in an online manner. 

Each agent will independently analyze daily Open High Low Close (OHLC) data sequences and develop a ranking of their belief that a particular ticker will return at least the risk-free rate in the next step. This ranking is based on the probability $p_{a}$, which represents the probability that the ticker will beat the risk-free rate. Each agent will express this probability as a belief distribution, represented by the parameters of a [Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution).

Once the analysis is complete, we will ask the agents to provide us with their `top-N` tickers. Subsequently, we could use these picks to populate our portfolio $\mathcal{P}$.

## Tasks
* __Prerequisites__: We'll load the daily Open High Low Close (OHLC) dataset and select tickers with complete data. We'll also compute the annualized daily return for each ticker in that dataset.
* __Task 1__: Develop a `world` function that agents will use to evaluate the possible actions and set up the ticker picker agents
    * `TODO`: Agents sample the `world`. If the ticker `XYZ` returns greater than or equal to the risk-free rate in the sample period, the agent receives a reward of `+1`. Otherwise, the agent receives a reward of `0`.
    * `TODO`: Initialize a collection of `N` ticker picker agents with an apriori belief that ticker `XYZ` is `good` or `bad`
* __Task 2__: In this task, we explore how the ticker picker agents learn which ticker symbols are better or worse bets.
    * `TODO`: Starting with a single ticker picker agent and a collection of `K` potential ticker symbols, we feed the agent daily (annualized) logarithmic return data and watch it evolve its belief system as new data is added.
* __Task 3__: Expand the single agent analysis to `N` agents, each exploring `K` potential ticker symbols.
    * `TODO`: After the agents complete their training, analyze their findings and have each agent compute its `top-N` ticker picks.  

## Setup
Several external `Julia` packages enable the computations in this example. To load the required packages and any custom codes the teaching team has developed to work with these packages, we [include](https://docs.julialang.org/en/v1/manual/code-loading/) the `Include.jl` file:

In [3]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git`
   Resolving package versions...
    Updating `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Project.toml`
  [4119e0bf] ~ VLQuantitativeFinancePackage v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main` ⇒ v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main`
    Updating `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2/Manifest.toml`
  [4119e0bf] ~ VLQuantitativeFinancePackage v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main` ⇒ v1.0.0-DEV `https://github.com/varnerlab/VLQuantitativeFinancePackage.jl.git#main`
Precompiling project...
  ✓ VLQuantitativeFinancePackage
  1 dependency successfully precompiled in 3 seconds. 268 already precompiled.
  Activating project at `~/Desktop/julia_work/CHEME-130-eCornell-Repository/courses/CHEME-136/module-2`
 

## Prerequisites: Load and clean the historical dataset
We gathered daily open-high-low-close data for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) from `01-03-2018` until `12-29-2023`, along with data for a few exchange-traded funds and volatility products during that time. We'll use this data in the subsequent study, but we must load and clean it up first. In particular, in the `prerequisites` code block, we will:
* First, we'll load the data and remove tickers that do not have the maximum number of trading days. We'll store the cleaned data in the `dataset::Dict{String, DataFrame}` variable.
* Next, we'll get a list of all the tickers in the `dataset,` sort them alphabetically, and store them in the `all_tickers_array` variable.
* Finally, we'll compute the logarithmic return for tickers in the `dataset` using the `log_return_matrix(...)` function and store these values in the `all_firms_log_return_matrix` variable (assuming a `252`-day trading year)

In [5]:
original_dataset = MyMarketDataSet() |> x-> x["dataset"];

### Clean the data
Not all of the tickers in our dataset have the maximum number of trading days for various reasons, e.g., acquistion or de-listing events. Let's collect only those tickers with the maximum number of trading days.

* First, let's compute the number of records for a company that we know has a maximim value, e.g., `AAPL` and save that value in the `maximum_number_trading_days` variable:

In [7]:
maximum_number_trading_days = original_dataset["AAPL"] |> nrow

1508

Now, lets iterate through our data and collect only those tickers that have `maximum_number_trading_days` records. Save that data in the `dataset::Dict{String,DataFrame}` variable:

In [9]:
dataset = Dict{String,DataFrame}();
for (ticker,data) ∈ original_dataset
    if (nrow(data) == maximum_number_trading_days)
        dataset[ticker] = data;
    end
end
dataset;

Let's get a list of firms that we have in cleaned up `dataset`, and save it in the `all_tickers_array` array:

In [11]:
all_tickers_array = keys(dataset) |> collect |> sort;

Finally, let's compute the annualized logarithmic growth rate time series for each ticker in `dataset` using the `log_return_matrix(...)` function:
> The `log_return_matrix(dataset::Dict{String, DataFrame}, firms::Array{String,1}; Δt::Float64 = (1.0/252.0), risk_free_rate::Float64 = 0.0) -> Array{Float64,2}` takes the `dataset` argument (which holds `N`-price values and other data for a ticker), and a `K`-dimensional list of ticker symbols, and returns the annualized growth rate $\mu$ time-series as `N-1`$\times$`K` array, where the rows correspond to times, and the columns correspond to tickers.

In [13]:
all_firms_log_growth_matrix = log_growth_matrix(dataset, all_tickers_array, 
    Δt = (1.0/252.0), risk_free_rate = 0.0);

## __Task 1__: Initialize the `world` model and set up a ticker picker agent
The world function takes action `a`, i.e., chooses a `ticker` from the collection of stocks/ETFs that we are exploring, and asks whether or not the return of this `ticker` at the end of today was greater or equal to the risk-free rate. 
* If the daily return is greater than or equal to the risk-free rate, the `agent` receives a reward of `r = 1`. Otherwise, the `agent` receives a `r = 0` reward.

### TODO: Setup a ticker-picker agent model

First, let's specify the tickers that we want to examine in the `tickers` array, and store the number of tickers in the `K` variable:

In [17]:
tickers = ["MRK", "JNJ", "MET", "NFLX", "AAPL", "AMD", "MU", "INTC", "MSFT", "SPY", "SPYD", "MMM",
    "UNH", "JPM", "OXY", "TSLA", "PEP", "LMT", "CMCSA", "ECL", "SRE", "BAC", "C", "WFC", "QQQ", "KR", "NOC", "GS"];
K = length(tickers);

Next, we construct the `EpsilonSamplingModel` instance which holds information about the ϵ-greedy sampling approach. The `EpsilonSamplingModel` type has one additional field, the `ϵ` field which controls the approximate fraction of `exploration` steps the algorithm takes; `exploration` steps are purely random.

In [19]:
sampling_model = build(MyEpsilonSamplingBanditModel, (
    K = K, # number bandit arms
    α = ones(K), # initialize to uniform values
    β = ones(K), # initialize to uniform values
    ϵ = 0.3, # fraction of random steps
));

### TODO: Setup the world model instance
Fill me in.

In [64]:
function world(action::Int64, start::Int64, data::Dict{String, DataFrame}, tickers::Array{String,1}; 
        Δt::Float64 = (1.0/252.0), risk_free_rate::Float64 = 0.05, buffersize::Int64 = 1)::Int64

    # initialize -
    result_flag = 0;
    T = buffersize*Δt;

    # grab the ticker we are looking at?
    ticker_symbol = tickers[action];
    
    # compute the expected return over the horizon -
    price_df = data[ticker_symbol];
    time = range(start+1,(start+buffersize), step=1) |> collect
    buffer = Array{Float64,1}();
    for t ∈ time
        P₁ = price_df[t-1,  :volume_weighted_average_price]
        P₂ = price_df[t, :volume_weighted_average_price]
        R = (1/Δt)*log(P₂/P₁); # growth rate
        push!(buffer,R);
    end
    μ = mean(buffer);

    # if we invested 1 USD, in each how much would we have at the end of horizon -
    W_risk_free = exp(risk_free_rate*T);
    W_ticker = exp(μ*T);
    
    # are we better or worse relative to the risk-free investment?
    if (W_ticker >= W_risk_free)
        result_flag = 1;
    end

    # default -
    return result_flag;
end;

Set the `risk_free_rate` variable:

In [23]:
risk_free_rate = 0.047;

Fill me in

In [25]:
world_model = build(MyTickerPickerWorldModel, (
    tickers = tickers,
    data = dataset,
    world = world,
    risk_free_rate = risk_free_rate
));

## __Task 2__: Run a single `ticker-picker` agent and explore its preferences
Let's run a single `ticker-picker` agent and examine what it returns using the `sample(...)` function. 
* The `sample(...)` function takes the agent model `model::EpsilonSamplingModel`, the `world::Function`, the cleaned `dataset` and your list of `tickers`, along with the `horizon` parameter, i.e., how many iterations we want the search to run for,  and the `risk_free_rate`. 
* The `sample(...)` function returns a dictionary holding the $(\alpha,\beta)$ parameters for each ticker (values) for iteration (keys).  

In [27]:
time_sample_results_dict_eps = VLQuantitativeFinancePackage.sample(sampling_model, world_model; 
    horizon = (maximum_number_trading_days - 1))

Dict{Int64, Vector{Beta}} with 1507 entries:
  1144 => [Beta{Float64}(α=32.0, β=26.0), Beta{Float64}(α=34.0, β=32.0), Beta{F…
  1175 => [Beta{Float64}(α=32.0, β=26.0), Beta{Float64}(α=34.0, β=32.0), Beta{F…
  719  => [Beta{Float64}(α=32.0, β=26.0), Beta{Float64}(α=34.0, β=32.0), Beta{F…
  1028 => [Beta{Float64}(α=32.0, β=26.0), Beta{Float64}(α=34.0, β=32.0), Beta{F…
  699  => [Beta{Float64}(α=32.0, β=26.0), Beta{Float64}(α=34.0, β=32.0), Beta{F…
  831  => [Beta{Float64}(α=32.0, β=26.0), Beta{Float64}(α=34.0, β=32.0), Beta{F…
  1299 => [Beta{Float64}(α=32.0, β=26.0), Beta{Float64}(α=34.0, β=32.0), Beta{F…
  1438 => [Beta{Float64}(α=32.0, β=26.0), Beta{Float64}(α=34.0, β=32.0), Beta{F…
  1074 => [Beta{Float64}(α=32.0, β=26.0), Beta{Float64}(α=34.0, β=32.0), Beta{F…
  319  => [Beta{Float64}(α=32.0, β=26.0), Beta{Float64}(α=34.0, β=32.0), Beta{F…
  687  => [Beta{Float64}(α=32.0, β=26.0), Beta{Float64}(α=34.0, β=32.0), Beta{F…
  1199 => [Beta{Float64}(α=32.0, β=26.0), Beta{Float64}(α=34.0, 

In [58]:
Z = time_sample_results_dict_eps[20]

28-element Vector{Beta}:
 Beta{Float64}(α=32.0, β=26.0)
 Beta{Float64}(α=34.0, β=32.0)
 Beta{Float64}(α=9.0, β=14.0)
 Beta{Float64}(α=18.0, β=19.0)
 Beta{Float64}(α=18.0, β=20.0)
 Beta{Float64}(α=56.0, β=35.0)
 Beta{Float64}(α=2.0, β=6.0)
 Beta{Float64}(α=7.0, β=12.0)
 Beta{Float64}(α=29.0, β=22.0)
 Beta{Float64}(α=7.0, β=10.0)
 Beta{Float64}(α=4.0, β=9.0)
 Beta{Float64}(α=16.0, β=18.0)
 Beta{Float64}(α=28.0, β=24.0)
 ⋮
 Beta{Float64}(α=15.0, β=15.0)
 Beta{Float64}(α=6.0, β=13.0)
 Beta{Float64}(α=10.0, β=9.0)
 Beta{Float64}(α=8.0, β=11.0)
 Beta{Float64}(α=40.0, β=33.0)
 Beta{Float64}(α=28.0, β=25.0)
 Beta{Float64}(α=6.0, β=10.0)
 Beta{Float64}(α=47.0, β=37.0)
 Beta{Float64}(α=70.0, β=45.0)
 Beta{Float64}(α=18.0, β=20.0)
 Beta{Float64}(α=2.0, β=7.0)
 Beta{Float64}(α=9.0, β=13.0)

## __Task 3__: Run a collection of `N` ticker-picker agents and examine their preferences
Repeat the single-agent analysis with `N` agents by running the `sample(...)` method inside a `for` loop. We'll store the results of the last time point in the `agent_specific_data::Array{Beta,2}(undef, K, number_of_agents)` array. 
* The `agent_specific_data` array holds the `Beta` distributions for each agent, i.e., it holds the preferences for each agent (cols) for each `ticker` in our collection (rows).

In [66]:
number_of_agents = 10000;
trading_day_index = 1479
agent_specific_data = Array{Beta,2}(undef, K, number_of_agents);

for agent_index ∈ 1:number_of_agents
    
    # sample -
    time_sample_results_dict_eps = VLQuantitativeFinancePackage.sample(sampling_model, world_model; 
        horizon = (maximum_number_trading_days - 1));
    #beta_array = build_beta_array(time_sample_results_dict_eps[trading_day_index]);

    # grab data for this agent -
    for k = 1:K
        agent_specific_data[k, agent_index] = time_sample_results_dict_eps[trading_day_index][k]
    end
end

In [67]:
agent_specific_data[1,:] # data for each agent for ticker 1 for all the agents

10000-element Vector{Beta}:
 Beta{Float64}(α=15.0, β=19.0)
 Beta{Float64}(α=12.0, β=15.0)
 Beta{Float64}(α=17.0, β=17.0)
 Beta{Float64}(α=13.0, β=11.0)
 Beta{Float64}(α=27.0, β=25.0)
 Beta{Float64}(α=28.0, β=25.0)
 Beta{Float64}(α=9.0, β=12.0)
 Beta{Float64}(α=29.0, β=27.0)
 Beta{Float64}(α=6.0, β=11.0)
 Beta{Float64}(α=26.0, β=22.0)
 Beta{Float64}(α=15.0, β=16.0)
 Beta{Float64}(α=7.0, β=12.0)
 Beta{Float64}(α=6.0, β=11.0)
 ⋮
 Beta{Float64}(α=13.0, β=14.0)
 Beta{Float64}(α=35.0, β=27.0)
 Beta{Float64}(α=21.0, β=21.0)
 Beta{Float64}(α=29.0, β=24.0)
 Beta{Float64}(α=27.0, β=25.0)
 Beta{Float64}(α=6.0, β=10.0)
 Beta{Float64}(α=17.0, β=20.0)
 Beta{Float64}(α=12.0, β=14.0)
 Beta{Float64}(α=16.0, β=16.0)
 Beta{Float64}(α=42.0, β=38.0)
 Beta{Float64}(α=4.0, β=8.0)
 Beta{Float64}(α=2.0, β=7.0)

## The wisdom of the collective
Now that we have prefernces for the `N` agents (encoded as `Beta` distributions for each ticker), let's develop a concencous belief in which tickers to include in our portfolio $\mathcal{P}$. First, let's compute the agent-specific rank of eack ticker, where `rank = 1` is the best, and `rank = K` is the worst. We'll store these values in the `preference_rank_array` array.

In [70]:
preference_rank_array = Array{Int,2}(undef, number_of_agents, K);
for agent ∈ 1:number_of_agents
        
    # ask an agent about thier preference for ticker i -
    experience_distributions = agent_specific_data[:,agent]
    preference_vector = preference(experience_distributions, tickers) .|> x-> trunc(Int64,x) # trunc function is cool!
    
    # package -
    for i ∈ 1:K
        preference_rank_array[agent, i] = preference_vector[i];
    end
end
preference_rank_array

LoadError: UndefVarError: `tiedrank` not defined

### Compute the frequency dictionary
Let's count the times a `ticker` is ranked in the `top 10` across the `N` agents and then normalize by the number of agents, i.e., compute the frequency of being ranked in the `top 10`. We'll store this value in the `frequency_dictionary,` where the `ticker` is the key, and the frequency is the value.
* `Hypothesis`: Those `tickers` with high rank are likelier to beat an alternative `risk-free` investment, while low rank `tickers` do not outperform a `risk-free` alternative investment. 

In [35]:
frequency_dictionary = Dict{String,Float64}();
for i ∈ eachindex(tickers)
    
    # compute the frequency -
    freq = findall(x-> x ≤ 10, preference_rank_array[:,i]) |> x-> length(x) |> x-> x/number_of_agents
    
    # get the ticker -
    ticker = tickers[i];
    frequency_dictionary[ticker] = freq;
end

In [36]:
frequency_dictionary

Dict{String, Float64} with 28 entries:
  "MSFT"  => 0.88
  "JPM"   => 0.881
  "C"     => 0.8178
  "MRK"   => 0.6291
  "UNH"   => 0.8782
  "SPY"   => 0.8758
  "BAC"   => 0.8247
  "MU"    => 0.3878
  "SRE"   => 0.8689
  "TSLA"  => 0.8696
  "KR"    => 0.8181
  "AMD"   => 0.6148
  "INTC"  => 0.8836
  "MMM"   => 0.8734
  "NOC"   => 0.8277
  "ECL"   => 0.8802
  "LMT"   => 0.8836
  "GS"    => 0.8276
  "OXY"   => 0.8784
  "CMCSA" => 0.8732
  "JNJ"   => 0.6479
  "SPYD"  => 0.8778
  "NFLX"  => 0.5511
  "WFC"   => 0.8323
  "PEP"   => 0.8802
  ⋮       => ⋮

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.